# Analyis of grid hyperparameter search

In [ ]:
import json
import pathlib
import pandas as pd

In [ ]:
fname = "runs/experiment03/all_metrics.json"

with open(pathlib.Path(fname)) as f:
    all_metrics = json.load(f)

In [ ]:
import collections.abc

def flatten_dict_of_dicts(d, parent_key=''):
    items = []
    for k, v in d.items():
        new_key = parent_key + (k,) if parent_key else (k,)
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten_dict_of_dicts(v, new_key).items())
        else:
            items.append((new_key, v))
    return dict(items)


flatten_dict_of_dicts({
    "a": {'a1': {'a2': 1, 'a3': 2}},
    "b": {'b1': {'b2': 3, 'b3': 4}}
        })

In [ ]:
data = {}
for k, run_metrics in all_metrics.items():
    data[k] = flatten_dict_of_dicts(run_metrics)

pd.DataFrame.from_dict(data)